In [1]:
!pip install --user sklearn
!pip install --user iisignature
!pip install --user tslearn

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle kernels output ruslankl/eeg-data-analysis -p /path/to/dest'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np
import pandas as pd 
import os
import random
from tqdm import tqdm
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from scipy.stats import mannwhitneyu
import iisignature as sig

init_notebook_mode(connected=True) ## plotly init
seed = 123
random.seed = seed

In [3]:
#data import
filenames_list = os.listdir('../input/Alcoholics/SMNI_CMI_TRAIN/Train') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file
print(len(filenames_list))
for file_name in tqdm(filenames_list):
    temp_df = pd.read_csv('../input/Alcoholics/SMNI_CMI_TRAIN/Train/' + file_name) ## read from the file to df
    EEG_data = EEG_data.append(temp_df) ## add the file data to the main df
    
EEG_data = EEG_data.drop(['Unnamed: 0'], axis=1) ## remove the unused column
EEG_data.loc[EEG_data['matching condition'] == 'S2 nomatch,', 'matching condition'] =  'S2 nomatch' ## remove comma sign from stimulus name

In [4]:
#example try for create train set
temp_df = pd.read_csv('../input/Alcoholics/SMNI_CMI_TRAIN/Train/Data1.csv')
print(temp_df['subject identifier'][0])
print(temp_df["matching condition"][0])
print(np.array(temp_df["sensor value"]).reshape([64,256]))


In [5]:
#S1: S1 obj - a single object shown;
s1 = 0
#S12: S2 nomatch - object 2 shown in a non matching condition (S1 differed from S2)
s12 = 0
#S21: S2 match - object 2 shown in a matching condition (S1 was identical to S2),
s21 = 0
s1_X_train = np.zeros((160,256,64))
s1_y_train = np.zeros(160)
s21_X_train = np.zeros((159,256,64))
s21_y_train = np.zeros(159)
s12_X_train = np.zeros((149,256,64))
s12_y_train = np.zeros(149)


classifier = {'a':1,'c':0}

filenames_list = os.listdir('../input/Alcoholics/SMNI_CMI_TRAIN/Train') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file
print(len(filenames_list))
for file_name in tqdm(filenames_list):
    temp_df = pd.read_csv('../input/Alcoholics/SMNI_CMI_TRAIN/Train/' + file_name) ## read from the file to df
    if temp_df["matching condition"][0] == "S1 obj":
        s1_X_train[s1] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
        s1_y_train[s1] = classifier[temp_df['subject identifier'][0]]
        s1 += 1
    if temp_df["matching condition"][0] == "S2 match":
        s21_X_train[s21] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
        s21_y_train[s21] = classifier[temp_df['subject identifier'][0]]
        s21 += 1
    if temp_df["matching condition"][0] == "S2 nomatch,":
        s12_X_train[s12] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
        s12_y_train[s12] = classifier[temp_df['subject identifier'][0]]
        s12 += 1
print(s1)
print(s21)
print(s12)


In [6]:
#t1: S1 obj - a single object shown;
t1 = 0
#t12: S2 nomatch - object 2 shown in a non matching condition (S1 differed from S2)
t12 = 0
#t21: S2 match - object 2 shown in a matching condition (S1 was identical to S2),
t21 = 0
t1_X_test = np.zeros((160,256,64))
t1_y_test = np.zeros(160)
t21_X_test = np.zeros((160,256,64))
t21_y_test = np.zeros(160)
t12_X_test = np.zeros((160,256,64))
t12_y_test = np.zeros(160)


classifier = {'a':1,'c':0}

filenames_list = os.listdir('../input/Alcoholics/SMNI_CMI_TEST') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file
print(len(filenames_list))
for file_name in tqdm(filenames_list):
    if file_name == "Test":
        pass
    else:
        temp_df = pd.read_csv('../input/Alcoholics/SMNI_CMI_TEST/' + file_name) ## read from the file to df
        if temp_df["matching condition"][0] == "S1 obj":
            t1_X_test[t1] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
            t1_y_test[t1] = classifier[temp_df['subject identifier'][0]]
            t1 += 1
        if temp_df["matching condition"][0] == "S2 match":
            t21_X_test[t21] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
            t21_y_test[t21] = classifier[temp_df['subject identifier'][0]]
            t21 += 1
        if temp_df["matching condition"][0] == "S2 nomatch,":
            t12_X_test[t12] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
            t12_y_test[t12] = classifier[temp_df['subject identifier'][0]]
            t12 += 1
print(t1)
print(t21)
print(t12)


In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tqdm import tqdm
for i in tqdm(range(3,20)):
    knn = KNeighborsTimeSeriesClassifier(n_neighbors=i, n_jobs=12)
    knn.fit(s1_X_train, s1_y_train)
    print(knn.score(t1_X_test,t1_y_test))

In [ ]:
from tslearn.svm import TimeSeriesSVC
clf = TimeSeriesSVC(C=1.0, kernel="gak")
clf.fit(s1_X_train, s1_y_train)
print(clf.score(t1_X_test, t1_y_test))

In [ ]:
s1_sig_train = sig.sig(s1_X_train, 2)
s1_sig_test = sig.sig(t1_X_test, 2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using Random Forest")
plt.show()
score = accuracy_score(t1_y_test, y_pred)
print("score is " + str(score))


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using Logistic Regression")
plt.show()
score = accuracy_score(t1_y_test, y_pred)
print("score is " + str(score))


In [ ]:
from sklearn.svm import SVC
clf = SVC(probability=True)
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using SVC")
plt.show()
score = accuracy_score(t1_y_test, y_pred)
print("score is " + str(score))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
for i in tqdm(range(3,20)):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(s1_sig_train, s1_y_train)
    y_pred = knn.predict(s1_sig_test)
    print(accuracy_score(t1_y_test, y_pred))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using AdaBoostClassifier")
plt.show()
score = accuracy_score(t1_y_test, y_pred)
print("score is " + str(score))

In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tqdm import tqdm
for i in tqdm(range(3,20)):
    knn = KNeighborsTimeSeriesClassifier(n_neighbors=i, n_jobs=12)
    knn.fit(s12_X_train, s12_y_train)
    print(knn.score(t12_X_test,t12_y_test))

In [ ]:
from tslearn.svm import TimeSeriesSVC
import datetime
start = datetime.datetime.now()
clf = TimeSeriesSVC(C=1.0, kernel="gak")
clf.fit(s12_X_train, s12_y_train)
score = clf.score(t12_X_test, t12_y_test)
end = datetime.datetime.now()
print(score)
print(end-start)

In [ ]:
s12_sig_train = sig.sig(s12_X_train, 2)
s12_sig_test = sig.sig(t12_X_test, 2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
start = datetime.datetime.now()
clf = RandomForestClassifier()
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using Random Forest")
plt.show()
score = accuracy_score(t12_y_test, y_pred)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)


In [ ]:
from sklearn.linear_model import LogisticRegression
start = datetime.datetime.now()
clf = LogisticRegression()
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using LogisticRegression")
plt.show()
score = accuracy_score(t12_y_test, y_pred)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)

In [ ]:
from sklearn.svm import SVC
start = datetime.datetime.now()
clf = SVC(probability = True)
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using SVC")
plt.show()
score = accuracy_score(t12_y_test, y_pred)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
for i in tqdm(range(3,20)):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(s12_sig_train, s12_y_train)
    y_pred = knn.predict(s12_sig_test)
    print(accuracy_score(t12_y_test, y_pred))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
start = datetime.datetime.now()
clf = AdaBoostClassifier()
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)

from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
# Visualisation with plot_metric
from sklearn.metrics import accuracy_score
bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using AdaBoostClassifier")
plt.show()
score = accuracy_score(t12_y_test, y_pred)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)